# ETL PROCESSES

# Developing the ETL process for each of the tables.

In [1]:
# Imports
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
# Connectiong to the database
conn = psycopg2.connect("host=127.0.0.1 dbname=data_eng user=franchise password='420GLOCKzone.'")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root, '*.json'))
        for f in files:
            all_files.append(os.path.abspath(f))

    return all_files

# Process song_data

In [4]:
# Creating the songs and artists dimensional tables.
song_files = get_files('./data/song_data/')
filepath = song_files[0]

In [5]:
# ETL on a single song and load the record into each table.
df = pd.read_json(filepath, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARDNS031187B9924F0,32.67828,-83.22295,Georgia,Tim Wilson,SONYPOM12A8C13B2D7,I Think My Wife Is Running Around On Me (Taco ...,186.48771,2005


In [6]:
# Extracting data from the songs table
song_data_df = df[['song_id', 'title','artist_id','year','duration']]
song_data_values = song_data_df.values   # Returns a numpy representation of values in the dataframe
first_record_df = song_data_values[0]
song_data = first_record_df.tolist()
song_data


['SONYPOM12A8C13B2D7',
 'I Think My Wife Is Running Around On Me (Taco Hell)',
 'ARDNS031187B9924F0',
 2005,
 186.48771]

In [7]:
# Inserting the record into the sng table
cur.execute(song_table_insert, song_data)
conn.commit()

UndefinedTable: relation "songs" does not exist
LINE 2:     INSERT INTO songs
                        ^
